# BLS series selector - Tool 2 of 3 for accessing ATUS files

## This notebook will allow you to take the dictonary you generated in the BLS Web Scraper and interactivly select demographics to get the series IDs.

## The BLS API requires a list of series IDs to download the required files. You can also use this list here:

[BLS Series Report] (https://data.bls.gov/cgi-bin/srgate)

As a warning the output of the Series Report tool is a collection of individual excel files or one csv file with multiple sections for each series. This is very difficult for Python or Excel to read in. You can use the BLS API notebook to clean and merge your files. 

### This notebook assumes that you have your dictonary file in the/data directory. If you cloned or downloaded this repository, there is an example version already there.

For a nice online resource about widgets see: [widgets](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916)

In [1]:
#Import modules
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [2]:
#This opens the dictonary file made from the tu.series file. I have included a sample in the data directory
#It needs to have the *_text columns to work.
lexicon = pd.read_csv('data/with_replace.csv', dtype=object) #we will need everything to be strings to match later

In [3]:
#if you want to check
lexicon.head(5)

,Unnamed: 0,series_id,sex_code,race_code,educ_code,age_code,pertype_code,series_title,earn_code,actcode_code,...,actcode_text,age_text,earn_text,educ_text,lfstat_text,orig_text,pertype_text,race_text,sex_text,stattype_text
0,0,TUU10100AA01000007,0,0,0,0,0,Number of persons - Employed,0,0,...,"Total, all activities",15 years and over,All persons,All education levels,Employed,NaN,All days,All races,Both sexes,Number of persons (in thousands)
1,1,TUU10100AA01000013,0,0,0,0,0,"Number of persons - Employed, Multiple jobholders",0,0,...,"Total, all activities",15 years and over,All persons,All education levels,Employed,NaN,All days,All races,Both sexes,Number of persons (in thousands)
2,2,TUU10100AA01000014,0,0,0,0,0,"Number of persons - Employed, Single jobholders",0,0,...,"Total, all activities",15 years and over,All persons,All education levels,Employed,NaN,All days,All races,Both sexes,Number of persons (in thousands)
3,3,TUU10100AA01000015,0,0,0,0,0,"Number of persons - Employed, Wage and salary ...",0,0,...,"Total, all activities",15 years and over,All persons,All education levels,Employed,NaN,All days,All races,Both sexes,Number of persons (in thousands)
4,4,TUU10100AA01000018,0,0,0,0,0,"Number of persons - Employed, Self-employed wo...",0,0,...,"Total, all activities",15 years and over,All persons,All education levels,Employed,NaN,All days,All races,Both sexes,Number of persons (in thousands)


# This section defines the widgets, currently there are a limited number of selections, but you can modify the code to match your needs. I indicate where to make the changes.

Note: This section needs to be optimized to make it more user friendly and customizable. If you fix something, let me know!

**If anything odd happens below you can re-initialize the widgets by running all the blocks from here down.**

In [4]:
#define the function to pull all the unique values from the df column we want to search
def unique_sorted_values_plus_ALL(array):
    """This function will pull all the unique values out of the code_text
    column and put them into a list. This list will be used to populate the
    dropdowns"""
    unique = array.unique().tolist()
    unique.sort()
    return unique

### This section defines all of the dropdowns. 

**You can modify them easily by replacing the * in the following with your column 

`unique_sorted_values_plus_ALL(lexicon.*_text)`

**Also replace the * in 

`value = '*'`  - Set this to the default value for your column, it is the code_text that matches the 00 value in the code_code column.

`description = '\*'` - Call the button whatever you want!

In [5]:
#make the dropdown to select units
dropdown_stats = widgets.Dropdown(options = unique_sorted_values_plus_ALL(lexicon.stattype_text), value = 'Average hours per day', description = 'Stat Type:') #put in the name of whatever file you are using for your dictonary 
dropdown_gender = widgets.Dropdown(options = unique_sorted_values_plus_ALL(lexicon.sex_text), value = 'Both sexes', description = 'Gender:') #put in the name of whatever file you are using for your dictonary 
dropdown_eth = widgets.Dropdown(options = unique_sorted_values_plus_ALL(lexicon.race_text), value = 'All races', description = 'Ethnicity:')
dropdown_edu = widgets.Dropdown(options = unique_sorted_values_plus_ALL(lexicon.educ_text), value = 'All education levels', description = 'Education: Only works with 25 or older in the Ages selector')
dropdown_age = widgets.Dropdown(options = unique_sorted_values_plus_ALL(lexicon.age_text), value = '15 years and over', description = 'Ages:')
dropdown_day = widgets.Dropdown(options = unique_sorted_values_plus_ALL(lexicon.pertype_text), value = 'All days', description = 'Days:')

Because we want to use these in the API we need to either copy them or save them. This line makes a new list that will save your selected series and can later be used to output them to a csv file or to the clipboard.

In [6]:
#making a variable to save the list
list_ser = pd.DataFrame()

In [7]:
#create an output variable for the widget
output = widgets.Output() #this will display the series names
list_output = widgets.Output() #this creates a list of the series ids

### This function sets up a common filter to combine all of your widget choices. 

**You can add aditional choices by adding them to the arguments section:

`common_filtering(stats, gender, eth, edu, age, day, your_col):`

**You will also need to add it to the end of the filter

`common_filter = lexicon[(lexicon['stattype_text'] == stats) & (lexicon['sex_text'] == gender) & (lexicon['race_text']== eth) & (lexicon['educ_text']== edu) & (lexicon['age_text']== age) & (lexicon['pertype_text']== day) & (lexicon['*_text'])
== your_col]`



In [8]:
def common_filtering(stats, gender, eth, edu, age, day):
    """Function sets up the dataframe filtering using the dropdown widgets
    outputs the series name to a display frame and the list of series ids"""
    global list_ser
    output.clear_output()
    list_output.clear_output()
    common_filter = lexicon[(lexicon['stattype_text'] == stats) & (lexicon['sex_text'] == gender) & (lexicon['race_text']== eth) & (lexicon['educ_text']== edu) & (lexicon['age_text']== age) & (lexicon['pertype_text']== day)]
    with output:
        display(common_filter.iloc[:,7]) 
    with list_output:
        list_ser = common_filter.iloc[:,1]
        list_ser = list_ser.tolist()
        display(len(common_filter))

### Creating the handlers for the dropdowns they can be changed this way

The input of the common_filtering() function needs to be in the order they defined in the code above.

`common_filtering(stats, gender, eth, edu, age, day, *your_col*):`

Add a new function for `*your_col*` and replace the following with the value of the argument preceding your new variable. For example below you would enter `dropdown_day.value` before `change.new`. The `change.new` should be last, as your new column is the last argument for the function.

`def dropdown_day_eventhandler(change):
    common_filtering(dropdown_stats.value, dropdown_gender.value, dropdown_eth.value, dropdown_edu.value , dropdown_age.value, **enter the value of the argument before *your_col*, change.new)`



In [9]:
#create the handler to get all the rows for the selected values
def dropdown_stats_eventhandler(change):
    common_filtering(change.new, dropdown_gender.value, dropdown_eth.value, dropdown_edu.value, dropdown_age.value, dropdown_day.value )
def dropdown_gender_eventhandler(change):
    common_filtering(dropdown_stats.value, change.new, dropdown_eth.value, dropdown_edu.value, dropdown_age.value, dropdown_day.value)
def dropdown_eth_eventhandler(change):
    common_filtering(dropdown_stats.value, dropdown_gender.value, change.new , dropdown_edu.value, dropdown_age.value, dropdown_day.value)
def dropdown_edu_eventhandler(change):
    common_filtering(dropdown_stats.value, dropdown_gender.value, dropdown_eth.value, change.new, dropdown_age.value, dropdown_day.value)
def dropdown_age_eventhandler(change):
    common_filtering(dropdown_stats.value, dropdown_gender.value, dropdown_eth.value, dropdown_edu.value, change.new, dropdown_day.value)
def dropdown_day_eventhandler(change):
    common_filtering(dropdown_stats.value, dropdown_gender.value, dropdown_eth.value, dropdown_edu.value , dropdown_age.value, change.new)

## To make sure that the dropdown list "hears" your changes you need to bind them to the hander.

To make changes add a new line and replace the * with the values for your_col

`dropdown_*your_col*.observe(dropdown_*your_col*_eventhandler, names='value')`


In [10]:
#bind the handler to the dropdown
dropdown_stats.observe(dropdown_stats_eventhandler, names='value')
dropdown_gender.observe(dropdown_gender_eventhandler, names='value')
dropdown_eth.observe(dropdown_eth_eventhandler, names='value')
dropdown_edu.observe(dropdown_edu_eventhandler, names='value')
dropdown_age.observe(dropdown_age_eventhandler, names='value')
dropdown_day.observe(dropdown_day_eventhandler, names='value')

## Add a new line for your dropdown

To make changes add a new line and replace the * with the values for your_col

`display(dropdown_*your_col*)`

A few warnings about the data. Some demographics are in one one set of series. For example "Asian" cannot be mixed with any other factors or no results will be found. However "Black" can be found with either "All sexes". "Men" or "Women". Educational traits can only be used with '25 years or older' in Age or no records will be found. I have set the defaults for each sections to the most general traits.
>To see the results make sure you run the cell below the widgets.

In [11]:
#this window will display the outputs
display(dropdown_stats)
display(dropdown_gender)
display(dropdown_eth)
display(dropdown_edu)
display(dropdown_age)
display(dropdown_day)
#make your selections!

Dropdown(description='Stat Type:', options=('Average hours per day', 'Average hours per day for participants i…

Dropdown(description='Gender:', options=('Both sexes', 'Men', 'Women'), value='Both sexes')

Dropdown(description='Ethnicity:', options=('All races', 'Asian', 'Black or African American', 'White'), value…

Dropdown(description='Education: Only works with 25 or older in the Ages selector', index=1, options=('Advance…

Dropdown(description='Ages:', index=2, options=('15 to 19 years', '15 to 24 years', '15 years and over', '18 y…

Dropdown(description='Days:', options=('All days', 'Nonholiday weekdays', 'Weekdays', 'Weekend days and holida…

## The output below will display the results of your search. 

You should try to find a smallish number of series. API registration key holders are limited to 50 series per search. The BLS API notebook can quickly pull and merge as many series as you need so you can be more specific here. Once you a number you like, move to the next cell to save it to a csv file for the API.

In [12]:
print('The number of series selected is:')
display(list_output)
print('If this number is lower then the API limit of 50 then move to the next cell to copy the list for use in the API.')
pd.set_option('display.max_colwidth', 1000)
display(output)

The number of series selected is:


Output()

If this number is lower then the API limit of 50 then move to the next cell to copy the list for use in the API.


Output()

## If you have the API open, you can print the list below and simply copy and paste it into the json code. Otherwise use the new_or_append function to create a new csv or append this search to an existing csv file. 

In [44]:
#save your list or copy it to the clipboard
#copy
print(list_ser)

['TUU10101AA01005684            ', 'TUU10101AA01006341            ', 'TUU10101AA01008860            ', 'TUU10101AA01009412            ', 'TUU10101AA01010405            ', 'TUU10101AA01011205            ', 'TUU10101AA01012018            ', 'TUU10101AA01012366            ', 'TUU10101AA01012683            ', 'TUU10101AA01013268            ', 'TUU10101AA01013622            ', 'TUU10101AA01013988            ', 'TUU10101AA01014273            ', 'TUU10101AA01014547            ', 'TUU10101AA01015041            ', 'TUU10101AA01015617            ', 'TUU10101AA01015956            ', 'TUU10101AA01016313            ', 'TUU10101AA01016391            ']


## This function can make a new file if this is your first request or can append it to an existing file if you want to add to an existing file. Just remember the 50 limit. 
> Don't worry that the series IDs are now seperate from their descriptions. The BLS API notebook will put everything back together for you!

**The choice argument takes only two strings `'new'` to make a new csv or `'append'` to append to an existing csv. By default the file is made or expected to exist in the /data directory.**


In [49]:
filename = 'data/List_for_API.csv' #define the name of your file here

def new_or_append(choice, filename = filename):
    """Function creates either a new csv file or appends the list to 
    an existing file. choice options are 'new' or 'append'. The filename is defined
    in a variable or can be entered manually"""
    if choice == 'new':
        list_out = pd.DataFrame(list_ser)
        list_out.to_csv(filename)
    elif choice == 'append':
        list_out = pd.DataFrame(list_ser)
        list_out.to_csv(filename, mode = 'a', header = False)
        test_append = pd.read_csv(filename, index_col=0).reset_index(drop=True)
        test_append.to_csv(filename, mode = 'w', header = False)

In [50]:
#Run the function
new_or_append('new')